In [ ]:
! pip install -qU keras pandas scikit-learn tensorflow matplotlib pillow

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Image dimensions
img_width, img_height = 224, 224

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Directories for training and validation data
train_data_dir = 'C:\\Users\\Cody\\Desktop\\Projects\\Mr-Beans\\notebook\\exploration\\beans\\data\\quality grade\\train'
validation_data_dir = 'C:\\Users\\Cody\\Desktop\\Projects\\Mr-Beans\\notebook\\exploration\\beans\\data\\quality grade\\test'

# Load the datasets
train_dataset = datasets.ImageFolder(train_data_dir, transform=data_transforms['train'])
validation_dataset = datasets.ImageFolder(validation_data_dir, transform=data_transforms['validation'])

# Create the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8, shuffle=False)


In [10]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Additional pooling to further reduce dimensionality
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Calculate the new size after pooling
        self.fc1 = nn.Linear(128 * 14 * 14, 512)  # Reduced size from additional pooling (14x14)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, len(train_dataset.classes))  # Adjust based on number of classes

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = self.pool(nn.ReLU()(self.conv3(x)))
        x = self.pool2(x)  # Additional pooling layer for reducing dimensions
        x = x.view(-1, 128 * 14 * 14)  # Adjust for the reduced spatial dimensions
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model
model = CNNModel()


In [11]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10
verbose = True  # Set to True for detailed output

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    # Print epoch header
    if verbose:
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 30)
        
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Verbose output for each batch
        if verbose and batch_idx % 10 == 0:  # Display every 10 batches
            print(f'Batch {batch_idx + 1}/{len(train_loader)} - Loss: {loss.item():.4f}')
    
    # Average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}')
    
    # Validate the model
    model.eval()
    correct = 0
    total = 0
    running_val_loss = 0.0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(validation_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Verbose output for each validation batch
            if verbose and batch_idx % 5 == 0:  # Display every 5 validation batches
                print(f'Validation Batch {batch_idx + 1}/{len(validation_loader)} - Loss: {loss.item():.4f}')
    
    val_loss = running_val_loss / len(validation_loader)
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

print('Training completed.')


Epoch 1/10
------------------------------
Batch 1/10 - Loss: 1.0990
Epoch [1/10], Loss: 1.1729
Validation Batch 1/5 - Loss: 1.0243
Validation Loss: 0.9645, Validation Accuracy: 50.00%
Epoch 2/10
------------------------------
Batch 1/10 - Loss: 0.9362
Epoch [2/10], Loss: 0.7523
Validation Batch 1/5 - Loss: 0.0122
Validation Loss: 0.6327, Validation Accuracy: 70.00%
Epoch 3/10
------------------------------
Batch 1/10 - Loss: 0.6459
Epoch [3/10], Loss: 0.3984
Validation Batch 1/5 - Loss: 0.0003
Validation Loss: 0.7095, Validation Accuracy: 67.50%
Epoch 4/10
------------------------------
Batch 1/10 - Loss: 0.1883
Epoch [4/10], Loss: 0.2936
Validation Batch 1/5 - Loss: 0.0000
Validation Loss: 0.7102, Validation Accuracy: 67.50%
Epoch 5/10
------------------------------
Batch 1/10 - Loss: 0.3935
Epoch [5/10], Loss: 0.1932
Validation Batch 1/5 - Loss: 0.0001
Validation Loss: 0.9504, Validation Accuracy: 77.50%
Epoch 6/10
------------------------------
Batch 1/10 - Loss: 0.1164
Epoch [6/10]

In [6]:
# Define the path where you want to save the model
model_save_path = 'C:\\Users\\Cody\\Desktop\\Projects\\Mr-Beans\\notebook\\exploration\\beans\\models\\grade_cnn_model.pth'

# Save the model's state_dict after training is completed
torch.save(model.state_dict(), model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to C:\Users\Cody\Desktop\Projects\Mr-Beans\notebook\exploration\beans\models\grade_cnn_model.pth
